# Pipeline

### 1. Define ROI

1. 1stLevel_defROI: using all game1 runs of each subject to execute F-test analysis(cos(6*theta) + sin(6*theta))
2. 2ndLeven_defROI: using adult/high performance/all subjects data to execute 2nd one-sample t-test
3. makeMask: find maximum activities voxels of 2nd t-test map in previous ROI to define target ROIs(EC and vmPFC)

